In [ ]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 16.7 MB/s eta 0:00:00


In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("mrm8488/codebert-base-finetuned-stackoverflow-ner")
model = AutoModelForTokenClassification.from_pretrained("mrm8488/codebert-base-finetuned-stackoverflow-ner")

In [ ]:
from transformers import pipeline
import re

In [39]:
import pandas as pd
df = pd.read_json("/content/drive/MyDrive/Colab Notebooks/shell/datafinal.json")
df

,ID,Text,ContainsCode,CodeList
0,1,Test-driven development (TDD) is a software de...,,
1,2,Software development is an exciting field that...,False,
2,3,The development process often starts with a co...,,
3,4,Another important aspect of software developme...,True,git commit -m 'Initial commit'
4,5,"In the world of software development, language...",False,
...,...,...,...,...
1903,1904,"Software development, an integral part of any ...",True,List<String> names = new ArrayList<String>();
1904,1905,"In the world of software development, one of t...",False,
1905,1906,Web development is a popular field in software...,,
1906,1907,C++ is a general-purpose programming language ...,,


In [67]:
#query=df['Text'].iloc[86]
#query

In [41]:
result=pipeline("ner",model=model,tokenizer=tokenizer,grouped_entities=True)
#result(query)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/token_classification.py:169: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="simple"` instead.
  warnings.warn(


[{'entity_group': 'Language',
  'score': 0.9975115,
  'word': ' Python',
  'start': 588,
  'end': 594},
 {'entity_group': 'Data_Structure',
  'score': 0.9637265,
  'word': ' list',
  'start': 618,
  'end': 622},
 {'entity_group': 'Function',
  'score': 0.8374735,
  'word': ' sorted()',
  'start': 672,
  'end': 680},
 {'entity_group': 'Code_Block',
  'score': 0.9944212,
  'word': ' numbers =',
  'start': 700,
  'end': 709},
 {'entity_group': 'Code_Block',
  'score': 0.7720094,
  'word': ' [',
  'start': 710,
  'end': 711},
 {'entity_group': 'Value',
  'score': 0.89039683,
  'word': '5, 1, 9,',
  'start': 711,
  'end': 719},
 {'entity_group': 'Value',
  'score': 0.9640157,
  'word': ' 3, 7',
  'start': 720,
  'end': 724},
 {'entity_group': 'Code_Block',
  'score': 0.97299397,
  'word': '];',
  'start': 724,
  'end': 726},
 {'entity_group': 'Code_Block',
  'score': 0.99273205,
  'word': ' sorted_numbers = sorted(numbers);',
  'start': 727,
  'end': 760}]

In [42]:
df['ner_result']=' '
for i in range(df.shape[0]):
  query=df['Text'].iloc[i]
  output = result(query)
  if len(output)>0:
      k=[]
      m=[]
      for j in range(len(output)):
        if re.search('Code',output[j]['entity_group']):
          k.append(output[j]['word'])
        else:
          m=[]
        if re.search('Class',output[j]['entity_group']):
          if re.search('class|Class',output[j]['word']):
            k.append(output[j]['word'])
          else:
            k.append('class'+output[j]['word'])
        if re.search('Value',output[j]['entity_group']):
          if re.search('Hello|world|class',output[j]['word']):
            k.append(output[j]['word'])
  df['ner_result'].iloc[i]=k


<ipython-input-42-a308841370b9>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ner_result'].iloc[i]=k


In [43]:
df["ner_result"].fillna("",inplace=True)

In [45]:
df['ner_result_ext']=' '
for i in range(df.shape[0]):
  input_list=df["ner_result"].iloc[i]
  combined_text = ''.join(input_list)
  unique_letters = []
  for char in combined_text:
    if char not in unique_letters:
        unique_letters.append(char)

  df['ner_result_ext'].iloc[i]=unique_letters

<ipython-input-45-136b915b2a01>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ner_result_ext'].iloc[i]=unique_letters


In [85]:
df['ner_result_fin'] = df['ner_result_ext'].copy()

# Iterate through the rows of the DataFrame
for i in range(df.shape[0]):
    my_list = df["ner_result_fin"].iloc[i]

    # Check if the list contains only one element and that element is '\n'
    if len(my_list) == 1 and (my_list[0] == '\n' or my_list[0] == 'n'):
        my_list[0] = ''  # Replace '\n' with an empty string
    elif len(my_list) > 1:
        # Iterate through the elements of the list
        for j in range(len(my_list)):
            if my_list[j] == '\n'or my_list[j] == '\t':
                my_list[j] = ''  # Replace '\n' with an empty string

In [88]:
custom_classes = [
    ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6',
    '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
    'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '_', '`', 'a', 'b', 'c', 'd', 'e',
    'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}'
]

In [90]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer(classes=custom_classes)
s1 = df["ner_result_fin"]
t1 = mlb.fit_transform(s1)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:895: UserWarning: unknown class(es) ['', '^'] will be ignored
  warnings.warn(


In [91]:
submission = pd.DataFrame(t1)

In [92]:
excel_file_path = 'submission.xlsx'
#
submission.to_excel(excel_file_path, index=False)